<a href="https://colab.research.google.com/github/ykitaguchi77/GAN/blob/master/Toony_Yourself.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Toony Yourself**

implementation: http://cedro3.com/ai/toonify-yourself/#comment-869

original: 

In [7]:
#StyleGANをGit clone
%tensorflow_version 1.x
!git clone https://github.com/justinpinkney/stylegan2

%cd stylegan2
!nvcc test_nvcc.cu -o test_nvcc -run
 
# フォルダーの作成
!mkdir aligned
!mkdir generated

Cloning into 'stylegan2'...
remote: Enumerating objects: 269, done.
remote: Total 269 (delta 0), reused 0 (delta 0), pack-reused 269
Receiving objects: 100% (269/269), 2.32 MiB | 4.97 MiB/s, done.
Resolving deltas: 100% (141/141), done.
/content/stylegan2/stylegan2/stylegan2
CPU says hello.
GPU says hello.


In [8]:
# ffhqモデル(Baseモデル)とblendedモデルのダウンロード
import pretrained_networks
import gdown

 
# use my copy of the blended model to save Doron's download bandwidth
# get the original here https://mega.nz/folder/OtllzJwa#C947mCCdEfMCRTWnDcs4qw
#blended_url = "https://drive.google.com/uc?id=1H73TfV5gQ9ot7slSed_l-lim9X7pMRiU"
blended_url = "https://drive.google.com/uc?id=1BRqqHWk_4BjNHLXTrpHkoxmZhGw3CU59" 
ffhq_url = "http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-ffhq-config-f.pkl"

#エラーがでてうまくいかないので、ダイレクトにダウンロードすることにする 
#_, _, Gs_blended = pretrained_networks.load_networks(blended_url)
#_, _, Gs = pretrained_networks.load_networks(ffhq_url)
gdown.download(blended_url, quiet=False)
gdown.download(ffhq_url, quiet=False)


Access denied with the following error:



 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1BRqqHWk_4BjNHLXTrpHkoxmZhGw3CU59 

Downloading...
From: http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-ffhq-config-f.pkl
To: /content/stylegan2/stylegan2/stylegan2/stylegan2-ffhq-config-f.pkl
100%|██████████| 382M/382M [00:01<00:00, 255MB/s]


'stylegan2-ffhq-config-f.pkl'

#**必要な関数を定義**

In [9]:
# 画像表示
import matplotlib.pyplot as plt
from PIL import Image
import glob
import numpy as np

def display_pic(folder):
    fig = plt.figure(figsize=(30, 40))
    files = glob.glob(folder)
    files.sort()
    images=[]
    for i in range(len(files)):
        img = Image.open(files[i])    
        images = np.asarray(img)
        ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
        image_plt = np.array(images)
        ax.imshow(image_plt)
        ax.set_xlabel(str(i), fontsize=20)               
    plt.show()
    plt.close()  

# 潜在変数(latents)から生成した画像を表示
import PIL.Image
def display(latents):
    
    synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=False), minibatch_size=8)
  
    fig = plt.figure(figsize=(30, 40))   
    for i in range(len(latents)):
        vec = latents[i].reshape(1,18,512)
        images =  Gs_blended.components.synthesis.run(vec, randomize_noise=False, **synthesis_kwargs)  
        images = images.transpose((0,2,3,1))     
        PIL.Image.fromarray(images[0], 'RGB')   
        ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
        image_plt = np.array(images[0])
        ax.imshow(image_plt)
        ax.set_xlabel(str(i), fontsize=20)               
    plt.show()
    plt.close()

# 潜在変数(latents)の順番を指定して、トランジションのGIFを作成する
import os

def generate_gif(latents, idx):

    synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=False), minibatch_size=8)

    image_gif = []
    os.makedirs('my/gif', exist_ok=True)
    for j in range(len(idx)-1):
        for i in range(20):
            latent = latents[idx[j]]+(latents[idx[j+1]]-latents[idx[j]])*i/19
            latent = latent.reshape(1, 18, 512)
            images =  Gs_blended.components.synthesis.run(latent, randomize_noise=False, **synthesis_kwargs) 
            images = images.transpose((0,2,3,1)) 
            image_one = PIL.Image.fromarray(images[0], 'RGB')
            image_gif.append(image_one.resize((256,256))) 

    image_gif[0].save('./my/gif/anime.gif', save_all=True, append_images=image_gif[1:],
                      duration=100, loop=0) 

In [31]:
# サンプル写真のダウンロードと整形
import os 
import shutil
import requests


# rawフォルダを作成
try: 
    shutil.rmtree("/content/stylegan2/raw")
    print("directory deleted")
    print("")
except:
    pass
os.makedirs("/content/stylegan2/raw", exist_ok=True)


#Webから画像をダウンロードしてrawフォルダに格納
url = "https://images.unsplash.com/photo-1597223557154-721c1cecc4b0?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxzZWFyY2h8Mnx8aHVtYW4lMjBmYWNlfGVufDB8fDB8fA%3D%3D&w=1000&q=80"
file_name = "/content/stylegan2/raw/sample.jpg"

response = requests.get(url)
image = response.content
with open(file_name, "wb") as aaa:
    aaa.write(image)


directory deleted

aligned		       generated	       run_generator.py
align_images.py        grid_vid.py	       run_metrics.py
blend_models.py        LICENSE.txt	       run_projector.py
dataset_tool.py        metrics		       run_training.py
dnnlib		       pretrained_networks.py  stylegan2-ffhq-config-f.pkl
Dockerfile	       project_images.py       test_nvcc
docs		       projector.py	       test_nvcc.cu
fake_art_portrait.jpg  raw		       training
ffhq_dataset	       README.md


In [11]:


# 顔画像の切り取り
!python align_images.py raw aligned
display_pic('./aligned/*.png')

Traceback (most recent call last):
  File "align_images.py", line 33, in <module>
    for img_name in [x for x in os.listdir(RAW_IMAGES_DIR) if x[0] not in '._']:
FileNotFoundError: [Errno 2] No such file or directory: 'raw'


<Figure size 2160x2880 with 0 Axes>